# 1. Preparation

## 1.1 Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!cp -r /content/drive/MyDrive/Uni\ Leipzig/SS2022/.ssh ~/

## 1.2 Get correct CuDNN version
See https://github.com/googlecolab/colabtools/issues/2600



In [2]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8 libcudnn8-dev
2 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
Need to get 867 MB of archives.
After this operation, 1,867 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8-dev 8.4.1.50-1+cuda11.6 [447 MB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.4.1.50-1+cuda11.6 [420 MB]
Fetched 867 MB in 15s (57.8 MB/s)
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../libcudnn8-dev_8.4.1.50-1+cuda11.6_amd64.deb ..

## 1.3 Install necessary packages

In [3]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive'

/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive


In [4]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-3hzbc1_h
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-3hzbc1_h
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 1.3 MB 49.3 MB/s 
     |████████████████████████████████| 50 kB 7.5 MB/s 
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8602 sha256=74478ec81682bd02c8d91539179a77f625515dcd1116b67394bc35fe3cd3e410
  Stored in directory: /tmp/pip-ephem-wheel-cache-tl7pyh7c/wheels/ff/27/79/a53e92f32da44c8c4f51094ca156e6ff91103b7e48a341d62b
 

In [5]:
!python3 -m pip install -r 'requirements.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 362 kB 63.9 MB/s 
     |████████████████████████████████| 1.8 MB 38.4 MB/s 
     |████████████████████████████████| 419 kB 65.9 MB/s 
     |████████████████████████████████| 1.0 MB 51.1 MB/s 
     |████████████████████████████████| 115 kB 71.6 MB/s 
     |████████████████████████████████| 212 kB 70.5 MB/s 
     |████████████████████████████████| 141 kB 58.2 MB/s 
     |████████████████████████████████| 181 kB 61.4 MB/s 
     |████████████████████████████████| 157 kB 71.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 127 kB 56.9 MB/s 
     |████████████████████████████████| 157 kB 68.8 MB/s 
     |████████████████████████████████| 157 kB 78.4 MB/s 
     |████████████████████████████████| 156 kB 79.4 

In [18]:
import re

def para(model, tokenizer, row, anchor_col="sentence1", para_col="sentence2", label_col="label", path="glue", name="mrpc", split="train"):
  out_row = {"anchor": row[anchor_col],
             "para": row[para_col],
             "neg_1": "",
             "original_label": row[label_col],
             "contains_numeric": False,
             "path": path,
             "name": name,
             "split": split}

  # return samples which contain paraphrase as is
  if row[label_col] == 1:
        return out_row


  # move hard negative to neg_1
  out_row["neg_1"] = row[para_col]
  #mark rows with digits in anchor sentence, leads to logical errors in paraphrase
  if re.search("\d", row[anchor_col]):
        out_row["contains_numeric"] = True


  anchor = row[anchor_col]
  anchor = re.sub(r'\s([\.,!\?\'"])', r'\1', anchor)
  anchor = anchor.replace('" ', '"')


  encoding = tokenizer.encode("paraphrase: " + anchor, return_tensors="pt")
  input_ids = encoding.to(device)


  paraphrases = model.generate(
      input_ids=input_ids,
      max_length=128,
      early_stopping=True,
      num_beams=10,
      num_return_sequences=1

  )
  	
  para = tokenizer.decode(paraphrases[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)

  # remove first tokens from anchor and paraphrase, that contain "paraphrsedoutput: " and "paraphrase: "
  para_token_list =paraphrases[0].tolist()[7:]
  input_token_list = input_ids.tolist()[0][3:]

  # check if generated tokens are the same as in original sentence, which means sentence was not altered or just shuffeled.
  # Sentences like this are not used
  # remove paraphrases without new tokens and empty ones
  if not para or sorted(para_token_list) ==sorted(input_token_list):
    out_row["para"] = "None"
    return out_row
  else:
      para = para.replace("paraphrasedoutput: ", "")
      out_row["para"] = para
    
  return out_row

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
model.to(device)
print(" ")


In [19]:
import pandas as pd
dataset_definitions = [
    {"path": "glue", "name": "mrpc"},
    {"path": "paws", "name": "labeled_final"},
    {"path": "paws", "name": "labeled_swap"},
    {"path": "paws", "name": "unlabeled_final"}
]

out = []

for definition in dataset_definitions:
  dataset = datasets.load_dataset(path=definition["path"], name=definition["name"])

  for split in ["train", "validation", "test"]:
    if split not in dataset:
      continue
    dataset_part = pd.DataFrame(dataset[split].select(range(20)))
    #dataset_post = dataset_part.map(lambda x: para(model, tokenizer, x), num_proc=1)
    for idx, row in dataset_part.iterrows():
      out.append(para(model, tokenizer,row, split=split, path=definition["path"], name=definition["name"]))
out = pd.DataFrame(out)
out = out[out["para"] != "None"]
out.to_csv("out.csv")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]